# CODIGO DE RE-ENTRENAMIENTO SOFT

In [1]:
#Posibles paquetes necesarios para el desarrollo del proyecto. No harán falta todos siempre. Seleccionar los necesarios
import os
import numpy as np
import pandas as pd
import pickle
import cloudpickle #Mismas funciones de pickle de guardar modelos + funciones de guardado de las funciones específicasimport seaborn as sns
import sqlalchemy as sa
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline
%config IPCompleter.greedy = True

#Formato sin notación científica
pd.options.display.float_format = '{:15.2f}'.format 

#Automcompletar rápido
%config IPCompleter.greedy=True

#Desactivar los warnings
import warnings
warnings.filterwarnings("ignore")

#Representación visual de un pipeline
from sklearn import set_config
set_config(display = 'diagram') #diagram/text 

#Transformación de variables
from janitor import clean_names
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

#Modelos ML: CLASIFICACIÓN
from xgboost import XGBClassifier
#Métrica de error: Habrá que incluir la que necesitemos
from sklearn.metrics import roc_auc_score

# PRECIO_M2 IDEALISTA
#Ruta del proyecto
ruta_proyecto = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/007_AIRBNB'

#Nombre del fichero de datos
nombre_fichero_datos = 'precios_idealista_Dic24.csv'

#Cargar los datos
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos
precio_m2 = pd.read_csv(ruta_completa)


#Pipeline con las modificaciones hechas en la tabla
nombre_pipe = 'pipe_idealista.pickle'
ruta_pipe = ruta_proyecto + '/04_Modelos/' + nombre_pipe
with open(ruta_pipe, mode='rb') as file:
    pipe_idealista = cloudpickle.load(file)

precio_m2 = pipe_idealista.transform(precio_m2)

# LISTINGS AIRBNB
#Ruta del proyecto
ruta_proyecto = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/007_AIRBNB'

con = sa.create_engine('sqlite:///C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/007_AIRBNB/02_Datos/01_Originales/airbnb2025.db')

#Nombre del fichero de datos
nombre_fichero_datos = 'airbnb2025.db'

#Cargar los datos
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos
listings = pd.read_sql('listings', con).drop(columns='index')#.set_index('id')



# LISTINGS_DET AIRBNB
#Ruta del proyecto
ruta_proyecto = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/007_AIRBNB'

con = sa.create_engine('sqlite:///C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/007_AIRBNB/02_Datos/01_Originales/airbnb2025.db')

#Nombre del fichero de datos
nombre_fichero_datos = 'airbnb2025.db'

#Cargar los datos
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos
listings_det = pd.read_sql('listings_det', con).drop(columns='index')#.set_index('id')


#Pipeline con las modificaciones hechas en la tabla
nombre_pipe = 'pipe_preparacion_df.pickle'
ruta_pipe = ruta_proyecto + '/04_Modelos/' + nombre_pipe
with open(ruta_pipe, mode='rb') as file:
    pipe_preparacion_df = cloudpickle.load(file)

df = pipe_preparacion_df.transform((listings_det, listings, precio_m2))

#Pipeline con las modificaciones hechas en la tabla
nombre_pipe = 'pipe_calidad_datos.pickle'
ruta_pipe = ruta_proyecto + '/04_Modelos/' + nombre_pipe
with open(ruta_pipe, mode='rb') as file:
    pipe_calidad_datos = cloudpickle.load(file)
    
df = pipe_calidad_datos.transform(df)

#Pipeline con las modificaciones hechas en la tabla
nombre_pipe = 'pipe_gestion_variables.pickle'
ruta_pipe = ruta_proyecto + '/04_Modelos/' + nombre_pipe
with open(ruta_pipe, mode='rb') as file:
    pipe_gestion_variables = cloudpickle.load(file)

df = pipe_gestion_variables.transform(df)

id_df = df['id'].copy()
id_df.to_sql('id_df', con = con, if_exists = 'replace')

precio_total_df = df[['id', 'precio_total']].copy()
precio_total_df.to_sql('precio_total_df', con = con, if_exists = 'replace')

variables_finales = [
'accommodates',
'availability_30',
'availability_365',
'availability_90',
'bedrooms',
'bed',
'calculated_host_listings_count',
'has_availability',
'host_response_rate',
'host_response_time',
'instant_bookable',
'maximum_nights',
'neighbourhood_group',
'price',
'property_type',
'room_type',
'license',
'bathrooms',
'air',
'allowed',
'cleaning',
'hot water iron',
'iron',
'microwave',
'parking',
'refrigerator',
'shampoo',
'tv washer',
'wifi kitchen',
'neighbourhood_cleansed',
'and silverware',
'body',
'clothing',
'clothing storage',
'coffee',
'elevator',
'hair',
'host_identity_verified',
'host_verifications',
'kitchen essentials',
'allowed',
'basics',
'beds',
'dryer bed',
'hair dryer',
'maker',
'microwave hangers',
'oven',
'stove',
'review_scores_rating',
'review_scores_accuracy',
'review_scores_cleanliness',
'review_scores_checkin',
'review_scores_communication',
'review_scores_location',
'review_scores_value',
'host_is_superhost',
'bathrooms_text',
'precio_m2',
'latitude',
'longitude',
'amenities',
'number_of_reviews']


x = df[variables_finales].copy()
target = 'pisos_rentables'
y = df[target].copy()

nombre_pipe = 'pipe_entrenamiento.pickle'
ruta_pipe = ruta_proyecto + '/04_Modelos/' + nombre_pipe
with open(ruta_pipe, mode='rb') as file:
    pipe_entrenamiento = cloudpickle.load(file)

x = x.copy()
x.columns = [str(col) for col in x.columns]
pipe_ejecucion = pipe_entrenamiento.fit(x, y)   
nombre_pipe_ejecucion = 'pipe_ejecucion.pickle'
ruta_pipe_ejecucion = ruta_proyecto + '/04_Modelos/' + nombre_pipe_ejecucion
with open(ruta_pipe_ejecucion, mode='wb') as file:
   cloudpickle.dump(pipe_ejecucion, file)
pipe_ejecucion


Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('pipeline',
                                  Pipeline(steps=[('pipeline-1',
                                                   Pipeline(steps=[('functiontransformer',
                                                                    FunctionTransformer(func=<function calidad_datos at 0x0000024AF5183D80>))])),
                                                  ('pipeline-2',
                                                   Pipeline(steps=[('functiontransformer',
                                                                    FunctionTransformer(func=<function gestion_variables at 0x0000024AF5182C00>))]))])),
                                 ('columntransform...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.05,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=1000, n_jobs=-1,
                               num_parallel_tree=None, random_state=None, ...))])